In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab07.ipynb")

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
from Crypto.Hash import SHA256

# Lab 7: Digital Signatures
Contributions From: Ryan Cottone

Welcome to Lab 7! In this lab, we will learn about how asymmetric encryption is setup in the real world, and how digital signatures make this possible.

# Digital Signatures

Recall that Message Authentication Codes let us ensure messages cannot be *tampered* with (without us detecting it). However, MACs require a symmetric key -- what if we wanted to ensure integrity in an asymmetric environment? This is where digital signatures come in.

Formally, digital signatures are a collection of two functions:

$$\text{Sign}(SK, M)$$ 

which produces a signature on the message $M$ signed by the secret key $SK$, and 

$$\text{Verify}(PK, S, M)$$

which takes in a signature output by $\text{Sign}$ and returns True if $S$ corresponds to a valid signature from the owner of the public key $PK$ on the message $M$.

## RSA Signatures

One of the most basic signature schemes is that of textbook RSA signatures. In this scheme, Alice has an RSA keypair of $e$ as her public key and $d$ as her private key (exactly the same as RSA). She signs a message like:

$$\text{Sign}(d, M, N) = M^d \mod N$$

and one can verify the message is legitimate via:

$$\text{Verify}(e, S, M, N)= (S^e \stackrel{?}{=} M \mod N)$$

Notice that $S = M^d$, so $S^e \equiv (M^d)^e \equiv M^{ed} \equiv M \mod N$, as specified in RSA.

Note that this is insecure for reasons we will soon see!

**Question 1.1**: Implement the Textbook RSA Signature Scheme!

In [ ]:
# Finds M^d mod N. Use pow(base, exponent, modulus)
def sign(d, M, N):
    ...

# Finds S^e mod N and compares it to M. Use pow(base, exponent, modulus)
def verify(e, S, N, M):
    ...

In [ ]:
grader.check("q1_1")

Now that we've setup our (faulty) signature scheme, let's demonstrate a forgery attack on it! From the notes/lecture, you may know that we can set some $M \in [0, N-1]$ to be our signature, and our message will therefore be $M^e$.

$$S = M^d \implies M = S^e$$

Unfortunately, we can't choose the message to be whatever we want, but we can bruteforce it to some extent!

Below are some helper functions we will use.

In [ ]:
def getExpansion(n,m):
    arr = []
    
    while n > 0:
        r = n % m
        n //= m
        
        arr.append(r)
    
    return arr

def textToInt(s):
    total = 0
    
    for i in range(len(s)):
        total += ord(s[i])*(256**i)
    
    return total

def intToText(n):
    expansion = getExpansion(n, 256)
    
    finalStr = ""
    
    for i in range(len(expansion)):
        finalStr += chr(expansion[i])
        
    return finalStr

In [ ]:
N = 57827237212537328721403414859701117256950147043657179682429675635414974092267
p = 197392385385765730323593372879610033749
q = 292955764729855380124638330592302833983
e = 30976423168144014209716624790587982344466107184812898368404437795227984190221
d = 25506717026652416005253828955620679434349390414876094959093739255385916016069


# Pretend we are Mallory and don't know Alice's private key, but still want to forge a signature.
mock_signature = 111122223333344444

message = pow(mock_signature, e, N)
print('Derived message:', intToText(message))

print('\nChecking message...\n')
if verify(e, mock_signature, N, message):
    print('Message verified as being from Alice!')
else:
    print('Message failed signature verification!')

Of course, this is just gibberish, but since we were able to make the program think it was from Alice, we've broken the scheme entirely.

The correct way to build sign is: $$\text{Sign}(d, M) = H(M)^d \mod N$$
where $H$ is some cryptographic hash function. Verification is as following:

$$\text{Verify}(e, S, M, N)= (S^e \stackrel{?}{=} H(M) \mod N)$$

In [ ]:
## This code is given to you!

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def H(M):
    h = SHA256.new()
    h.update(bytes(int_to_bytes(M)))
    return int(h.hexdigest(), 16)

Let's rewrite our functions from earlier to be more secure:

In [ ]:
# Finds H(M)^d mod N.
def secureSign(d, M, N):
    return pow(H(M), d, N)

# Finds S^e mod N and compares it to H(M).
def secureVerify(e, S, N, M):
    return pow(S, e, N) == H(M)

If we try our attack from earlier, we see that it no longer works:

In [ ]:
mock_signature = 111122223333344444

message = pow(mock_signature, e, N)
print('Derived message:', intToText(message))

print('\nChecking message...\n')
if secureVerify(e, mock_signature, N, message):
    print('Message verified as being from Alice!')
else:
    print('Message failed signature verification!')

To make this work, we'd need to find some input $x$ to $H$ such that $H(x) = M$, which is hard under collision-resistant functions.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)